# Assignment 2

#### First Name: Maximilian
#### Last Name: von Klinski


## 1. Load Data from JSON

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Twitter App") \
    .getOrCreate()

In [ ]:
# Make sure to upload the corona_tweet_new.json file
df_twitter = spark.read.json("/content/drive/My Drive/_Universität_HPI/Semester 6/BigData/Assignments/assisgnment_2/corona_tweet_new.json")

In [ ]:
df_twitter.printSchema()

root
 |-- created_at: string (nullable = true)
 |-- favorite_count: long (nullable = true)
 |-- hashtags: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- id: string (nullable = true)
 |-- in_reply_to_status_id: string (nullable = true)
 |-- in_reply_to_user_id_str: string (nullable = true)
 |-- location: string (nullable = true)
 |-- reply_count: long (nullable = true)
 |-- retweet_count: long (nullable = true)
 |-- source: string (nullable = true)
 |-- user: struct (nullable = true)
 |    |-- contributors_enabled: boolean (nullable = true)
 |    |-- created_at: string (nullable = true)
 |    |-- default_profile: boolean (nullable = true)
 |    |-- default_profile_image: boolean (nullable = true)
 |    |-- description: string (nullable = true)
 |    |-- favourites_count: long (nullable = true)
 |    |-- follow_request_sent: string (nullable = true)
 |    |-- followers_count: long (nullable = true)
 |    |-- following: string (nullable = true)
 |    |-- frie

In [ ]:
### From the user nestec col select the following cols only id_str,followers_count,friends_count and created at
# (2 points)
from pyspark.sql.functions import col

# Here we used the selct function, friendscount is included in the task but not included in the example, for safety, I still included it and also location is not included in the example but we will later need it, so it is also included here
df_twitter=df_twitter.select(col("created_at"),
                             col("favorite_count"),
                             col("hashtags"),
                             col("id"),
                             col("in_reply_to_status_id"),
                             col("in_reply_to_user_id_str"),
                             col("location"),
                             col("reply_count"),
                             col("retweet_count"),
                             col("source"),
                             col("user.id_str").alias("user_id_str"),
                             col("user.followers_count").alias("user_followers_count"),
                             col("user.friends_count").alias("user_friends_count"),
                             col("user.created_at").alias("user_created_at")
                            )

In [ ]:
df_twitter.printSchema()

root
 |-- created_at: string (nullable = true)
 |-- favorite_count: long (nullable = true)
 |-- hashtags: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- id: string (nullable = true)
 |-- in_reply_to_status_id: string (nullable = true)
 |-- in_reply_to_user_id_str: string (nullable = true)
 |-- location: string (nullable = true)
 |-- reply_count: long (nullable = true)
 |-- retweet_count: long (nullable = true)
 |-- source: string (nullable = true)
 |-- user_id_str: string (nullable = true)
 |-- user_followers_count: long (nullable = true)
 |-- user_friends_count: long (nullable = true)
 |-- user_created_at: string (nullable = true)



In [ ]:
# Print the total count of number of records in df_twitter(1 point)
df_twitter.count()

15894

In [ ]:
# Extract the source lable from source col by droping the anchor tab and save it as another col named extracted_source
# for example <a href="https://mobile.twitter.com" rel="nofollow">Twitter Web App</a> => Twitter Web App
# you can use "<a [^>]+>([^<]+)" as regualr expresion and the group would be 1 for this regular expression.
#(4 points)
from pyspark.sql.functions import regexp_extract, col

df_twitter=df_twitter.withColumn("extracted_source", regexp_extract(col("source"), '<a [^>]+>([^<]+)', 1))
df_twitter.select(col('extracted_source'),col('source')).show()

+-------------------+--------------------+
|   extracted_source|              source|
+-------------------+--------------------+
|    Twitter Web App|<a href="https://...|
|Twitter for Android|<a href="http://t...|
|Twitter for Android|<a href="http://t...|
|Twitter for Android|<a href="http://t...|
|Twitter for Android|<a href="http://t...|
|    Twitter Web App|<a href="https://...|
| Twitter Web Client|<a href="http://t...|
|Twitter for Android|<a href="http://t...|
|Twitter for Android|<a href="http://t...|
| Twitter for iPhone|<a href="http://t...|
| Twitter for iPhone|<a href="http://t...|
|    Twitter Web App|<a href="https://...|
|Twitter for Android|<a href="http://t...|
|    Twitter Web App|<a href="https://...|
| Twitter for iPhone|<a href="http://t...|
|Twitter for Android|<a href="http://t...|
| Twitter for iPhone|<a href="http://t...|
|Twitter for Android|<a href="http://t...|
| Twitter for iPhone|<a href="http://t...|
|Twitter for Android|<a href="http://t...|
+----------

In [ ]:
# Convert the DataFrame into RDD
rdd_twitter=df_twitter.rdd.map(tuple)

In [ ]:
# Create a temporay table in memory with name as twitter (1 point)

df_twitter.createTempView("twitter")

## 2. Analyze Data

#### You will be writing code to find the answer to the questions listed below using Just RDD, Using spark SQL

- Analyze using RDD
- Analyze using Dataframe without temp table
- Analyze using spark.sql with temp table


#### 2.1 Get total number of unique users (1 point for each type)

In [ ]:
# Using RDD

num_unique_users_rdd = rdd_twitter.map(lambda tweet: (tweet[10], 1)).reduceByKey(lambda a, b: a + b).count()
print('The total number of unique users(RDD):',num_unique_users_rdd)

The total number of unique users(RDD): 14094


In [ ]:
# Using DataFrame
num_unique_users_df = df_twitter.select("user_id_str").distinct().count()
print('The total number of unique users(df):',num_unique_users_df)

The total number of unique users(df): 14094


In [ ]:
# Using spark.sql and the temporay table.
spark.sql('SELECT COUNT(DISTINCT user_id_str) AS Total_Number_of_unique_users_sparkSQL FROM twitter').show()

+-------------------------------------+
|Total_Number_of_unique_users_sparkSQL|
+-------------------------------------+
|                                14094|
+-------------------------------------+



#### 2.2 Get count of user who have more than 1 tweet in the data (2 points)

In [ ]:
# Using RDD
num_users_with_more_than_one_rdd = rdd_twitter.map(lambda tweet: (tweet[10], 1)).reduceByKey(lambda a, b: a + b).filter(lambda user: user[1] > 1).count()
print('Number of users with more than 1 tweet(RDD):',num_users_with_more_than_one_rdd)

Number of users with more than 1 tweet(RDD): 1016


In [ ]:
# Using DataFrame
num_users_with_more_than_one_df = df_twitter.groupby("user_id_str").count().filter("count > 1").count()
print('Number of users with more than 1 tweet(df):',num_users_with_more_than_one_df)

Number of users with more than 1 tweet(df): 1016


In [ ]:
# Using spark.sql and the temporay table.
spark.sql('SELECT COUNT(*) as number_of_users_with_more_than_one_tweet_sparkSQL FROM (SELECT COUNT(user_id_str) AS tweet_count FROM TWITTER GROUP BY user_id_str) WHERE tweet_count > 1').show()

+-------------------------------------------------+
|number_of_users_with_more_than_one_tweet_sparkSQL|
+-------------------------------------------------+
|                                             1016|
+-------------------------------------------------+



#### 2.3 Get total number unique extracted_source (1 point each)

In [ ]:
# Using RDD
num_unique_extracted_rdd = rdd_twitter.map(lambda tweet: (tweet[14], 1)).reduceByKey(lambda a, b: a + b).count()
print('Total number of unique extracted_sources(rdd):', num_unique_extracted_rdd)

Total number of unique extracted_sources(rdd): 133


In [ ]:
# Using DataFrame
num_unique_extracted_df =  df_twitter.select("extracted_source").distinct().count()
print('Total number of unique extracted_sources(df):', num_unique_extracted_df)

Total number of unique extracted_sources(df): 133


In [ ]:
# Using spark.sql and the temporay table.
spark.sql('SELECT COUNT(DISTINCT extracted_source) AS number_of_unique_extracted_sources_sparkSQL  FROM twitter').show()

+-------------------------------------------+
|number_of_unique_extracted_sources_sparkSQL|
+-------------------------------------------+
|                                        133|
+-------------------------------------------+



#### 2.4 Get top 5 most used extracted_source

In [ ]:
# Using RDD (5 points)
top_extracted_sources_rdd = rdd_twitter.map(lambda tweet: (tweet[14], 1)).reduceByKey(lambda a, b: a + b).takeOrdered(5, lambda s: -1 * s[1])
top_extracted_sources_rdd

[('Twitter for Android', 6262),
 ('Twitter for iPhone', 5698),
 ('Twitter Web App', 2878),
 ('Twitter for iPad', 428),
 ('Twitter Web Client', 136)]

In [ ]:
# Using DataFrame (2 points)
df_twitter.groupby("extracted_source").count().sort("count", ascending = False).show(5)

+-------------------+-----+
|   extracted_source|count|
+-------------------+-----+
|Twitter for Android| 6262|
| Twitter for iPhone| 5698|
|    Twitter Web App| 2878|
|   Twitter for iPad|  428|
| Twitter Web Client|  136|
+-------------------+-----+
only showing top 5 rows



In [ ]:
# Using spark.sql and the temporay table. (2 points)
spark.sql('SELECT extracted_source, COUNT(extracted_source) as count FROM twitter GROUP BY extracted_source ORDER BY count DESC LIMIT 5').show()

+-------------------+-----+
|   extracted_source|count|
+-------------------+-----+
|Twitter for Android| 6262|
| Twitter for iPhone| 5698|
|    Twitter Web App| 2878|
|   Twitter for iPad|  428|
| Twitter Web Client|  136|
+-------------------+-----+



#### 2.5 Get count of distinct hastags used ( 5 point each)

In [ ]:
# Using RDD
num_of_distinct_hastags_rdd = rdd_twitter.flatMap(lambda tweet: tweet[2]).distinct().count()
print('Number of distinct hashtags(rdd):', num_of_distinct_hastags_rdd)

Number of distinct hashtags(rdd): 1215


In [ ]:
from pyspark.sql.functions import explode

# Using DataFrame
num_of_distinct_hastags_df = df_twitter.select(explode(col('hashtags'))).distinct().count()
print('Number of distinct hashtags(df):', num_of_distinct_hastags_df)

Number of distinct hashtags(df): 1215


In [ ]:
# Using spark.sql and the temporay table.
spark.sql('SELECT COUNT(*) AS number_of_distinct_hashtags_sparkSQL FROM (SELECT DISTINCT EXPLODE(hashtags) FROM twitter)').show()

+------------------------------------+
|number_of_distinct_hashtags_sparkSQL|
+------------------------------------+
|                                1215|
+------------------------------------+



#### 2.6 Get top 5 hashtags

In [ ]:
# Using RDD (4 points)
top_5_hashtags_rdd = rdd_twitter.flatMap(lambda tweet: tweet[2]).map(lambda hashtags: (hashtags, 1)).reduceByKey(lambda a, b: a + b).takeOrdered(5, lambda s: -1 * s[1])
for  hashtag, count in top_5_hashtags_rdd:
  print(hashtag, ' count: ', count)

طبق_القدرات_للثانويه_ياريس  count:  385
Corona  count:  319
OilPrice  count:  251
COVID19  count:  125
corona  count:  123


In [75]:
# Using DataFrame (2 points)
top_5_hashtags_df = df_twitter.select(explode(col('hashtags'))).groupby("col").count().sort("count", ascending = False).show(5)

+--------------------+-----+
|                 col|count|
+--------------------+-----+
|طبق_القدرات_للثان...|  385|
|              Corona|  319|
|            OilPrice|  251|
|             COVID19|  125|
|              corona|  123|
+--------------------+-----+
only showing top 5 rows



In [74]:
# Using spark.sql and the temporay table. (2 points)
spark.sql('SELECT hashtag, COUNT(hashtag) as count FROM (SELECT EXPLODE(hashtags) as hashtag FROM twitter) GROUP BY hashtag ORDER BY count DESC LIMIT 5').show()

+--------------------+-----+
|             hashtag|count|
+--------------------+-----+
|طبق_القدرات_للثان...|  385|
|              Corona|  319|
|            OilPrice|  251|
|             COVID19|  125|
|              corona|  123|
+--------------------+-----+



#### 2.6 Get total number of tweets which are retweeted more than 100 times

In [76]:
# Using RDD
num_more_than_hundred_rdd = rdd_twitter.filter(lambda tweet: tweet[8] > 100).count()
print('Number of tweets retweeted more than 100 times(rdd):', num_more_than_hundred_rdd)

Number of tweets retweeted more than 100 times(rdd): 15753


In [77]:
# Using DataFrame
num_more_than_hundred_df = df_twitter.filter("retweet_count > 100").count()
print('Number of tweets retweeted more than 100 times(df):', num_more_than_hundred_df)

Number of tweets retweeted more than 100 times(df): 15753


In [80]:
# Using spark.sql and the temporay table.
spark.sql('SELECT COUNT(id) AS number_of_tweets_with_more_than_100_retweets_sparkSQL FROM twitter WHERE retweet_count > 100').show()

+-----------------------------------------------------+
|number_of_tweets_with_more_than_100_retweets_sparkSQL|
+-----------------------------------------------------+
|                                                15753|
+-----------------------------------------------------+



#### 2.8 Get top 3 most retweeted tweets per country (8 points)

In [109]:
import numpy as np
# Using RDD

top_3_retweeted_tweets_rdd = rdd_twitter.map(lambda tweet: (tweet[6],[tweet[3],tweet[8]])).groupByKey().mapValues(lambda x: np.sort(list(x),axis=0)[-3:]).collect()
top_3_retweeted_tweets_rdd

[('India',
  array([['1252336674580791297', '9973'],
         ['1252336682713649152', '9976'],
         ['1252336686362738689', '9988']], dtype='<U21')),
 ('USA',
  array([['1252336697825689601', '9982'],
         ['1252336698274562053', '9987'],
         ['1252336700170424322', '9994']], dtype='<U21')),
 ('China',
  array([['1252336667735863296', '999'],
         ['1252336671917510660', '9993'],
         ['1252336676778545152', '9998']], dtype='<U21')),
 ('Pakistan',
  array([['1252336687037837312', '9973'],
         ['1252336695778762752', '9975'],
         ['1252336698320658436', '9988']], dtype='<U21')),
 ('Mexico',
  array([['1252336683644747776', '9971'],
         ['1252336690284204032', '9994'],
         ['1252336699230892040', '9998']], dtype='<U21')),
 ('UK',
  array([['1252336669778272256', '9985'],
         ['1252336677395197953', '9989'],
         ['1252336683753730057', '9991']], dtype='<U21')),
 ('Spain',
  array([['1252336686773800963', '9969'],
         ['12523366991216

In [100]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number

# Using DataFrame
window_spec_df = Window.partitionBy("location").orderBy(col("retweet_count").desc())
df_with_row_number_df = df_twitter.withColumn("local_rank", row_number().over(window_spec_df))
top_retweeted_tweets_df = df_with_row_number_df.filter(col("local_rank") <= 3)

top_retweeted_tweets_df.select('location','id','retweet_count', 'local_rank').show(n=33)

+--------+-------------------+----------+-------------+
|location|                 id|local_rank|retweet_count|
+--------+-------------------+----------+-------------+
|  Canada|1252335430323888128|         1|         9997|
|  Canada|1252254877939531776|         2|         9992|
|  Canada|1252252082825986051|         3|         9987|
|   Chile|1252253612140490759|         1|         9988|
|   Chile|1252334891951427585|         2|         9984|
|   Chile|1252253710182481920|         3|         9978|
|   China|1252335780707684352|         1|         9998|
|   China|1252253596516843520|         2|         9993|
|   China|1252255562525560832|         3|         9984|
| Germany|1252334028092399622|         1|         9999|
| Germany|1252330902325248000|         2|         9997|
| Germany|1252252295510855682|         3|         9990|
|   India|1252332114948874240|         1|         9988|
|   India|1252252336921206787|         2|         9976|
|   India|1252254519116746754|         3|       

In [105]:
# Using spark.sql and the temporay table.
spark.sql('''SELECT location, id, retweet_count, rank AS local_rank FROM (SELECT *, RANK() OVER (PARTITION BY location ORDER BY retweet_count DESC) as rank FROM TWITTER) WHERE rank < 4 ORDER BY location, rank ASC''').show(n=33)

+--------+-------------------+-------------+----------+
|location|                 id|retweet_count|local_rank|
+--------+-------------------+-------------+----------+
|  Canada|1252335430323888128|         9997|         1|
|  Canada|1252254877939531776|         9992|         2|
|  Canada|1252252082825986051|         9987|         3|
|   Chile|1252253612140490759|         9988|         1|
|   Chile|1252334891951427585|         9984|         2|
|   Chile|1252253710182481920|         9978|         3|
|   China|1252335780707684352|         9998|         1|
|   China|1252253596516843520|         9993|         2|
|   China|1252255562525560832|         9984|         3|
| Germany|1252334028092399622|         9999|         1|
| Germany|1252330902325248000|         9997|         2|
| Germany|1252252295510855682|         9990|         3|
|   India|1252332114948874240|         9988|         1|
|   India|1252252336921206787|         9976|         2|
|   India|1252254519116746754|         9973|    

#### 2.9 Total number of tweets per country

In [114]:
# Using RDD (3 points)
country_number_dict_rdd = rdd_twitter.map(lambda tweet: tweet[6]).countByValue()

print('Number of tweets per country')
print('location : count')
for a in country_number_dict_rdd:
  print(a,':', country_number_dict_rdd[a])

Number of tweets per country
location : count
India : 1480
UK : 1376
Spain : 1464
USA : 1539
Canada : 1441
China : 1457
Pakistan : 1470
Germany : 1426
Mexico : 1409
Chile : 1410
Italy : 1422


In [115]:
# Using DataFrame (2 points)
df_twitter.groupBy("location").count().show()

+--------+-----+
|location|count|
+--------+-----+
| Germany| 1426|
|   India| 1480|
|   China| 1457|
|   Chile| 1410|
|   Italy| 1422|
|   Spain| 1464|
|     USA| 1539|
|  Mexico| 1409|
|      UK| 1376|
|  Canada| 1441|
|Pakistan| 1470|
+--------+-----+



In [113]:
# Using spark.sql and the temporay table. (1 point)
spark.sql('SELECT location, COUNT(id) as count FROM twitter GROUP BY location').show()

+--------+-----+
|location|count|
+--------+-----+
| Germany| 1426|
|   India| 1480|
|   China| 1457|
|   Chile| 1410|
|   Italy| 1422|
|   Spain| 1464|
|     USA| 1539|
|  Mexico| 1409|
|      UK| 1376|
|  Canada| 1441|
|Pakistan| 1470|
+--------+-----+



## 3. Save Data

#### 3.1 save the data such that you have seperate folder per country (2 points)

In [139]:
# Using DataFrame
# For this the hashtags column first has to be casted to string, since arrays are not supported by csv
df_twitter.withColumn("hashtags", col("hashtags").cast("string")).write.partitionBy('location').csv("/content/drive/My Drive/_Universität_HPI/Semester 6/BigData/Assignments/assisgnment_2/saved_data_2",header = 'true')

In [ ]:
test_df = spark.read.csv("/content/drive/My Drive/_Universität_HPI/Semester 6/BigData/Assignments/assisgnment_2/saved_data_2/location=Germany/part-00000-25372ed3-9aeb-41f2-b7ef-5b7f4086de05.c000.csv",header='True')
test_df.show()

#### 3.2 Save the data as parquet files (1 points)

In [131]:
# Using DataFrame
df_twitter.write.partitionBy('location').parquet("/content/drive/My Drive/_Universität_HPI/Semester 6/BigData/Assignments/assisgnment_2/location.parquet")

In [138]:
# Confirming that the saving worked of the parquet files worked
test_parquetFile = spark.read.parquet("/content/drive/My Drive/_Universität_HPI/Semester 6/BigData/Assignments/assisgnment_2/location.parquet")
test_parquetFile.show()

+--------------------+--------------+--------------------+-------------------+---------------------+-----------------------+-----------+-------------+--------------------+-------------------+--------------------+------------------+--------------------+-------------------+--------+
|          created_at|favorite_count|            hashtags|                 id|in_reply_to_status_id|in_reply_to_user_id_str|reply_count|retweet_count|              source|        user_id_str|user_followers_count|user_friends_count|     user_created_at|   extracted_source|location|
+--------------------+--------------+--------------------+-------------------+---------------------+-----------------------+-----------+-------------+--------------------+-------------------+--------------------+------------------+--------------------+-------------------+--------+
|Mon Apr 20 15:01:...|          5209|                  []|1252251170057457665|                 NULL|                   NULL|       6258|         7927|<a h